In [1]:
dataDict = np.load('datainput_FinerGrid.npy')[()]
qp.printDictKeys(dataDict)
dataDict['kinCube'].shape, dataDict['potCube'].shape, dataDict['dipCUBE'].shape, dataDict['geoCUBE'].shape

dict_keys(['kinCube', 'potCube', 'dipCUBE', 'geoCUBE', 'phis', 'gams', 'thes'])


((29, 21, 29, 9, 3),
 (29, 21, 29, 8),
 (29, 21, 29, 3, 8, 8),
 (29, 21, 29, 15, 3))

In [2]:
gsm_phi_ind = dataDict['phis'].index('P000-000')
gsm_gam_ind = dataDict['gams'].index('P016-923')
gsm_the_ind = dataDict['thes'].index('P114-804')

gsm_phi_ind, gsm_gam_ind, gsm_the_ind

(14, 10, 14)

In [3]:
from quantumpropagator import labTranformA,fromLabelsToFloats

phiV, gamV, theV = fromLabelsToFloats(dataDict)

phis = labTranformA(dataDict['phis'])
gams = labTranformA(dataDict['gams'])
thes = labTranformA(dataDict['thes'])

phiL = len(phis)
gamL = len(gams)
theL = len(thes)

In [4]:
phis,phiV,phiV[0]-phiV[-1],phiV[0]-phiV[1],phiL

(array([-7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. ,
        -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,
         4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  7. ]),
 array([-0.07 , -0.065, -0.06 , -0.055, -0.05 , -0.045, -0.04 , -0.035,
        -0.03 , -0.025, -0.02 , -0.015, -0.01 , -0.005,  0.   ,  0.005,
         0.01 ,  0.015,  0.02 ,  0.025,  0.03 ,  0.035,  0.04 ,  0.045,
         0.05 ,  0.055,  0.06 ,  0.065,  0.07 ]),
 -0.14000000000000001,
 -0.0050000000000000044,
 29)

In [5]:
gams,gamV,gamV[0]-gamV[-1],gamV[0]-gamV[1],gamL

(array([ 13.846,  14.154,  14.462,  14.769,  15.077,  15.385,  15.692,
         16.   ,  16.308,  16.615,  16.923,  17.231,  17.538,  17.846,
         18.154,  18.462,  18.769,  19.077,  19.385,  19.692,  20.   ]),
 array([ 0.24165829,  0.2470339 ,  0.25240952,  0.25776768,  0.26314329,
         0.26851891,  0.27387707,  0.27925268,  0.28462829,  0.28998646,
         0.29536207,  0.30073768,  0.30609584,  0.31147146,  0.31684707,
         0.32222269,  0.32758085,  0.33295646,  0.33833208,  0.34369024,
         0.34906585]),
 -0.10740756216773104,
 -0.0053756140961425392,
 21)

In [6]:
thes,theV,theV[0]-theV[-1],theV[0]-theV[1], theL

(array([ 121.061,  120.615,  120.168,  119.721,  119.274,  118.827,
         118.38 ,  117.933,  117.486,  117.039,  116.592,  116.145,
         115.698,  115.251,  114.804,  114.358,  113.911,  113.464,
         113.017,  112.57 ,  112.123,  111.676,  111.229,  110.782,
         110.335,  109.888,  109.441,  108.994,  108.547]),
 array([ 1.05645652,  1.05256444,  1.04866363,  1.04476282,  1.04086201,
         1.0369612 ,  1.03306038,  1.02915957,  1.02525876,  1.02135795,
         1.01745714,  1.01355633,  1.00965552,  1.00575471,  1.0018539 ,
         0.99796181,  0.994061  ,  0.99016019,  0.98625938,  0.98235857,
         0.97845776,  0.97455695,  0.97065614,  0.96675533,  0.96285452,
         0.9589537 ,  0.95505289,  0.95115208,  0.94725127]),
 0.10920525129728531,
 0.0038920842319474236,
 29)

In [7]:
dataDict['potCube'].shape, dataDict['kinCube'].shape,

((29, 21, 29, 8), (29, 21, 29, 9, 3))

In [8]:
#dataDict['potCube'] = dataDict['potCube'][:,0:20,:]
#dataDict['kinCube'] = dataDict['kinCube'][:,0:20,:]
#dataDict['potCube'].shape, dataDict['kinCube'].shape, 

In [9]:
#gspotALL = dataDict['potCube']
#gspot = gspotALL[:,:,:,0]
#gspot2 = np.empty_like(gspot)
#gspotALL2 = np.zeros_like(gspotALL)
#
#print(gspot.shape,np.amin(gspot),np.amax(gspot),np.amax(gspot)-np.amin(gspot))#

#p_s = 0.01
#g_s = 100
#t_s = 100
#p0 = phiV[14] 
#g0 = gamV[10]
#t0 = theV[14]

#for pi, p in enumerate(phiV):
#    for gi, g in enumerate(gamV):
#        for ti,t in enumerate(theV):
#             gspot2[pi,gi,ti] = p_s*(p-p0)**2 + g_s*(g-g0)**2 + t_s*(t-t0)**2
#                
#gspotALL2[:,:,:,0] = gspot23

#dataDict['potCube'] = gspotALL2

#saveNew = True
#if saveNew:
#    np.save('datainput_FinerGrid_potential.npy', dataDict)

In [10]:
%matplotlib notebook
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact#, interactive, fixed, interact_manual
import ipywidgets as widgets

def do3dplot(xs,ys,zss):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    X,Y = np.meshgrid(ys,xs)
    #ax.plot_wireframe(X, Y, zss)
    ax.scatter(X, Y, zss)
    
def do3dplot2(X,Y,Z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    #ax.plot_wireframe(X, Y, Z)
    ax.scatter(X, Y, Z)
    


# Potentials

## K matrix elements

In [11]:
def kinElem(mkelem,deriv,phi):
    expl = '{}\'th order derivative coefficient in K matrix'.format(deriv)
    tis = ['Tpp','Tpg','Tpt','Tgp','Tgg','Tgt','Ttp','Ttg','Ttt']
    tisexpl = '{} -> '.format(tis[mkelem])
    print(tisexpl + expl)
    x = gams
    y = thes
    z = dataDict['kinCube'][phi,:,:,mkelem,deriv]
    do3dplot(x,y,z)

interact(kinElem, mkelem = widgets.IntSlider(min=0,max=8,step=1,value=0), deriv = widgets.IntSlider(min=0,max=2,step=1,value=0),phi = widgets.IntSlider(min=0,max=phiL-1,step=1,value=0));

interactive(children=(IntSlider(value=0, description='mkelem', max=8), IntSlider(value=0, description='deriv',…

# Double the grid

In [12]:
phiV,gamV,theV

(array([-0.07 , -0.065, -0.06 , -0.055, -0.05 , -0.045, -0.04 , -0.035,
        -0.03 , -0.025, -0.02 , -0.015, -0.01 , -0.005,  0.   ,  0.005,
         0.01 ,  0.015,  0.02 ,  0.025,  0.03 ,  0.035,  0.04 ,  0.045,
         0.05 ,  0.055,  0.06 ,  0.065,  0.07 ]),
 array([ 0.24165829,  0.2470339 ,  0.25240952,  0.25776768,  0.26314329,
         0.26851891,  0.27387707,  0.27925268,  0.28462829,  0.28998646,
         0.29536207,  0.30073768,  0.30609584,  0.31147146,  0.31684707,
         0.32222269,  0.32758085,  0.33295646,  0.33833208,  0.34369024,
         0.34906585]),
 array([ 1.05645652,  1.05256444,  1.04866363,  1.04476282,  1.04086201,
         1.0369612 ,  1.03306038,  1.02915957,  1.02525876,  1.02135795,
         1.01745714,  1.01355633,  1.00965552,  1.00575471,  1.0018539 ,
         0.99796181,  0.994061  ,  0.99016019,  0.98625938,  0.98235857,
         0.97845776,  0.97455695,  0.97065614,  0.96675533,  0.96285452,
         0.9589537 ,  0.95505289,  0.95115208,  0.9472

In [13]:
import quantumpropagator as qp

def doubleAxespoins(Y):
    N = len(Y)
    X = np.arange(0, 2*N, 2)
    X_new = np.arange(2*N-1)       # Where you want to interpolate
    Y_new = np.interp(X_new, X, Y) 
    return(Y_new)

phis_n = doubleAxespoins(phiV)
gams_n = doubleAxespoins(gamV)
thes_n = doubleAxespoins(theV)

pp,gg,tt = qp.fromFloatsToLabels(phis_n, gams_n, thes_n)
ppL = len(pp)
ggL = len(gg)
ttL = len(tt)

In [14]:
phis_n,gams_n,thes_n

(array([-0.07  , -0.0675, -0.065 , -0.0625, -0.06  , -0.0575, -0.055 ,
        -0.0525, -0.05  , -0.0475, -0.045 , -0.0425, -0.04  , -0.0375,
        -0.035 , -0.0325, -0.03  , -0.0275, -0.025 , -0.0225, -0.02  ,
        -0.0175, -0.015 , -0.0125, -0.01  , -0.0075, -0.005 , -0.0025,
         0.    ,  0.0025,  0.005 ,  0.0075,  0.01  ,  0.0125,  0.015 ,
         0.0175,  0.02  ,  0.0225,  0.025 ,  0.0275,  0.03  ,  0.0325,
         0.035 ,  0.0375,  0.04  ,  0.0425,  0.045 ,  0.0475,  0.05  ,
         0.0525,  0.055 ,  0.0575,  0.06  ,  0.0625,  0.065 ,  0.0675,  0.07  ]),
 array([ 0.24165829,  0.2443461 ,  0.2470339 ,  0.24972171,  0.25240952,
         0.2550886 ,  0.25776768,  0.26045548,  0.26314329,  0.2658311 ,
         0.26851891,  0.27119799,  0.27387707,  0.27656487,  0.27925268,
         0.28194049,  0.28462829,  0.28730737,  0.28998646,  0.29267426,
         0.29536207,  0.29804988,  0.30073768,  0.30341676,  0.30609584,
         0.30878365,  0.31147146,  0.31415927,  0.316847

In [ ]:
doublALL = np.zeros((ppL,ggL,ttL))
doublALL2 = np.zeros((ppL,ggL,ttL,8))
kineti = np.zeros((ppL,ggL,ttL,9,3))

p_s = 100
g_s = 100
t_s = 100
p0 = phis_n[28] 
g0 = gams_n[20]
t0 = thes_n[28]

for pi, p in qp.log_progress(enumerate(phis_n),every=1,size=(len(phis_n))):
    for gi, g in enumerate(gams_n):
        for ti,t in enumerate(thes_n):
            doublALL[pi,gi,ti] = p_s*(p-p0)**2 + g_s*(g-g0)**2 + t_s*(t-t0)**2
            kineti[pi,gi,ti] = qp.calc_g_G(p,g,t)
            
doublALL2[:,:,:,0] = doublALL

newdict = {}
newdict['kinCube'] = kineti
newdict['potCube'] = doublALL2
newdict['dipCUBE'] = np.zeros((ppL,ggL,ttL,3,8,8))
newdict['geoCUBE'] = np.zeros((ppL,ggL,ttL,3,15))
newdict['phis'] = pp
newdict['gams'] = gg
newdict['thes'] = tt

saveNew = True
if saveNew:
    np.save('doubledoublefinerArmonic.npy', newdict)
    
qp.printDict(newdict)

In [16]:
newdict['kinCube'].shape

(57, 41, 57, 9, 3)